In [ ]:
#Import necessary packages and import data to dataframes
import numpy as np 
import pandas as pd  
import os

import warnings
warnings.filterwarnings('ignore')

PATH = '../input/'
player_position = pd.read_csv(f'{PATH}player_punt_data.csv')
player_role = pd.read_csv(f'{PATH}play_player_role_data.csv')
play_information = pd.read_csv(f'{PATH}play_information.csv')
video_review = pd.read_csv(f'{PATH}video_review.csv')

In [ ]:
#Combine GameKey and PlayID into one column and then sort
player_role['game_playid'] = player_role['GameKey'].map(str) + "_" + player_role['PlayID'].map(str)
player_role = player_role.sort_values(by = ['game_playid', 'Role']).reset_index(drop=True)
player_role.head()

In [ ]:
#Make new dataframe that groups organizes by formation type 
play_formations = player_role
play_formations = play_formations.groupby(['Season_Year','game_playid','GameKey','PlayID'])['Role'].apply('-'.join).reset_index()
play_formations = play_formations.sort_values(by= ['GameKey','PlayID']).reset_index(drop=True)
play_formations.rename(columns={'Role':'formation'}, inplace=True)
play_formations.rename(str.lower, axis='columns', inplace=True)
print(play_formations.shape)
play_formations.head()

In [ ]:
#Add column that shows formation count and frequency
count = pd.value_counts(play_formations['formation'].values)
frequency = pd.value_counts(play_formations['formation'].values, normalize=True)
count = count.to_frame().reset_index()
frequency = frequency.to_frame().reset_index()
temp = pd.merge(count,frequency, on='index', how='left')
temp.columns = ['formation','formation_count','formation_frequency']
formations = pd.merge(play_formations,temp[['formation','formation_count','formation_frequency']], on='formation', how='left')
print(formations.shape)
formations.head()

In [ ]:
concussion_play = video_review[['GameKey', 'PlayID']]
concussion_play['game_playid'] = concussion_play['GameKey'].map(str) + "_" + concussion_play['PlayID'].map(str)
concussion_play = concussion_play.sort_values(by = ['GameKey','PlayID']).reset_index(drop=True)
concussion_play.head()

In [ ]:
#Combine the concussion play with formation type
concussion_temp = pd.merge(concussion_play,play_formations[['game_playid','formation']], on='game_playid', how='left')
concussion_temp['concussion_count'] = concussion_temp['formation'].map(concussion_temp['formation'].value_counts())
concussion_temp['concussion_frequency'] = concussion_temp['formation'].map(concussion_temp['formation'].value_counts(normalize=True))
concussion_temp = concussion_temp.sort_values(by=['concussion_count','formation'], ascending=False).reset_index(drop=True)
concussion_temp.head()

In [ ]:
formation_df = pd.merge(formations,concussion_temp[['game_playid', 'concussion_count','concussion_frequency']], on='game_playid', how='left')
formation_df = formation_df.sort_values(by=['concussion_frequency','formation_count'], ascending=False).fillna(0.00000).reset_index(drop=True)
print(formation_df.shape)
formation_df.head()

In [ ]:
formation_df.to_csv('formation_df.csv',index=False)

# Summary

This Notebook Creates a dataframe that groups certain formations and then provides statistical information alongside concussion risk for formation.